In [1]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
import torch.nn as nn
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
import torch
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
from util.node import Node
import shutil
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
import pickle
from util.func import get_patch_size
import random
from util.data import ModifiedLabelLoader
from tqdm import tqdm

Heatmaps showing where a prototype is found will not be generated because OpenCV is not installed.


In [2]:
# run_path = '/home/harishbabu/projects/PIPNet/runs/010-CUB-27-imgnet_OOD_cnext26_img=224_nprotos=20'
run_path = '/home/harishbabu/projects/PIPNet/runs/031-CUB-18-imgnet_cnext26_img=224_nprotos=20_orth-on-rel'
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

------------------------- No discretization -------------------------


In [3]:
print(root)

root
	052+004
		052+053
			cub_052_Pied_billed_Grebe
			053+051
				cub_053_Western_Grebe
				051+050
					cub_051_Horned_Grebe
					cub_050_Eared_Grebe
		004+086
			004+032
				cub_004_Groove_billed_Ani
				032+031
					cub_032_Mangrove_Cuckoo
					031+033
						cub_031_Black_billed_Cuckoo
						cub_033_Yellow_billed_Cuckoo
			086+045
				cub_086_Pacific_Loon
				045+100
					045+003
						cub_045_Northern_Fulmar
						003+001
							cub_003_Sooty_Albatross
							001+002
								cub_001_Black_footed_Albatross
								cub_002_Laysan_Albatross
					100+023
						100+101
							cub_100_Brown_Pelican
							cub_101_White_Pelican
						023+024
							cub_023_Brandt_Cormorant
							024+025
								cub_024_Red_faced_Cormorant
								cub_025_Pelagic_Cormorant



In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

ckpt_path = os.path.join(run_path, 'checkpoints', 'net_trained_last')
checkpoint = torch.load(ckpt_path, map_location=device)

# Obtain the dataset and dataloaders
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers, num_prototypes = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    num_prototypes=num_prototypes,
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
criterion = nn.NLLLoss(reduction='mean').to(device)

Num classes (k) =  18 ['cub_001_Black_footed_Albatross', 'cub_002_Laysan_Albatross', 'cub_003_Sooty_Albatross', 'cub_004_Groove_billed_Ani', 'cub_023_Brandt_Cormorant'] etc.
Classes:  {'cub_001_Black_footed_Albatross': 0, 'cub_002_Laysan_Albatross': 1, 'cub_003_Sooty_Albatross': 2, 'cub_004_Groove_billed_Ani': 3, 'cub_023_Brandt_Cormorant': 4, 'cub_024_Red_faced_Cormorant': 5, 'cub_025_Pelagic_Cormorant': 6, 'cub_031_Black_billed_Cuckoo': 7, 'cub_032_Mangrove_Cuckoo': 8, 'cub_033_Yellow_billed_Cuckoo': 9, 'cub_045_Northern_Fulmar': 10, 'cub_050_Eared_Grebe': 11, 'cub_051_Horned_Grebe': 12, 'cub_052_Pied_billed_Grebe': 13, 'cub_053_Western_Grebe': 14, 'cub_086_Pacific_Loon': 15, 'cub_100_Brown_Pelican': 16, 'cub_101_White_Pelican': 17}
Number of prototypes:  50


In [8]:
@torch.no_grad() 
def get_all_activations(net, projectloader, num_classes, device, foldername, args, k=10, node=None, wandb_logging=True):
    print(f"Visualizing prototypes for topk of node {node.name} ...", flush=True)

    # if projectloader.shuffle:
    #     raise('Disable shuffle of projection dataloader')

    name2label = projectloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(projectloader, node)
    projectloader = modifiedLabelLoader
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name

    dir = os.path.join(args.log_dir, foldername)
    if not os.path.exists(dir):
        os.makedirs(dir)

    near_imgs_dirs = dict()
    seen_max = dict()
    saved = dict()
    saved_ys = dict()
    tensors_per_prototype = dict()
    
    for p in range(net.module._num_prototypes):
        near_imgs_dir = os.path.join(dir, str(p))
        near_imgs_dirs[p]=near_imgs_dir
        seen_max[p]=0.
        saved[p]=0
        saved_ys[p]=[]
        tensors_per_prototype[p]=[]
        all_activations[p] = []
    
    patchsize, skip = get_patch_size(args)

    # imgs = projectloader.dataset.imgs
    imgs = projectloader.filtered_imgs
    
    # Make sure the model is in evaluation mode
    net.eval()
    # classification_weights = net.module._classification.weight
    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight

    # Show progress on progress bar
    img_iter = tqdm(enumerate(projectloader),
                    total=len(projectloader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    # Iterate through the data
    images_seen = 0
    topks = dict()
    # Iterate through the training set
    for i, (xs, orig_y, ys) in img_iter:
        images_seen+=1
        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            # Use the model to classify this batch of input data
            pfs, pooled, _ = net(xs, inference=True)
            pooled = pooled[node.name].squeeze(0) 
            pfs = pfs[node.name].squeeze(0) # pfs.shape -> [768, 26, 26] (after squeeze)
            
            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                if c_weight > 1e-3:#ignore prototypes that are not relevant to any class
                    if p not in topks.keys():
                        topks[p] = []
                    all_activations[p].append(pooled[p].item())
                        
                    # if len(topks[p]) < k:
                    #     topks[p].append((i, pooled[p].item()))
                    # else:
                    #     topks[p] = sorted(topks[p], key=lambda tup: tup[1], reverse=True)
                    #     if topks[p][-1][1] < pooled[p].item():
                    #         topks[p][-1] = (i, pooled[p].item())
                    #     if topks[p][-1][1] == pooled[p].item():
                    #         # equal scores. randomly chose one (since dataset is not shuffled so latter images with same scores can now also get in topk).
                    #         replace_choice = random.choice([0, 1])
                    #         if replace_choice > 0:
                    #             topks[p][-1] = (i, pooled[p].item())
    return all_activations

In [9]:
for node in root.nodes_with_children():
    all_activations = get_all_activations(net, projectloader, node.num_children(), device, f'visualised_prototypes_topk/{node.name}', args, node=node)
    # set weights of prototypes that are never really found in projection set to 0
    set_to_zero = []
    classification_layer = getattr(net.module, '_'+node.name+'_classification')
    # if topks:
    #     for prot in topks.keys():
    #         found = False
    #         for (i_id, score) in topks[prot]:
    #             if score > 0.1:
    #                 found = True
    #         if not found:
    #             torch.nn.init.zeros_(classification_layer.weight[:,prot])
    #             set_to_zero.append(prot)
    #     print(f"Weights of prototypes of node {node.name}", set_to_zero, "are set to zero because it is never detected with similarity>0.1 in the training set", flush=True)

NameError: name 'net' is not defined

In [ ]:
import traceback
from collections import defaultdict

# class_names = os.listdir(os.path.join(data_path, train_folder+'/'))
# class_names.sort()
# label2name = {i : name for (i,name) in enumerate(class_names)}
name2label = projectloader.dataset.class_to_idx
label2name = {label:name for name, label in name2label.items()}
dataloader = projectloader
# dataloader = train_loader

"""
Example of the structure of max_activations
root_prototype_vectors_0 -> {
                                cub_001_Black_footed_Albatross_lvl0 -> [], # list of activations
                                cub_011_Rusty_Blackbird_lvl0 -> [],
                                cub_060_Glaucous_winged_Gull_lvl1 -> []
                            }
"""
max_activations = defaultdict(lambda: defaultdict(list))

# accuracy of prediction at each node, maps each node.name to its accuracy
node_accuracy = defaultdict(lambda: {'n_examples': 0, 'n_correct': 0, 'accuracy': None, 'children': defaultdict(lambda: {'n_examples': 0, 'n_correct': 0})})

for i, (image, label) in enumerate(dataloader):
    input = image.cuda()
    target = label.cuda()

    batch_names = [label2name[y.item()] for y in label]     
    batch_size = len(target)

    batch_start = time.time()   

    num_parents_in_batch = 0

    with torch.no_grad():

        # _ = model.module(input) 
        
#         conv_features = model.module.conv_features(input)
        pfs, pooled, _ = net(xs, inference=True)
        for node in model.module.root.nodes_with_children():
            try:
#                 distances = model.module.prototype_distances(conv_features, node.name)
#                 min_distances = -nn.functional.max_pool2d(-distances,
#                                                         kernel_size=(distances.size()[2], distances.size()[3])) # global max pooling
#                 min_distances = min_distances.view(-1, getattr(model.module,"num_" + node.name + "_prototypes"))
#                 avg_distances = -nn.functional.avg_pool2d(-distances,
#                                                         kernel_size=(distances.size()[2], distances.size()[3])) # global avg pooling
#                 avg_distances = avg_distances.view(-1, getattr(model.module,"num_" + node.name + "_prototypes"))
#                 prototype_activations_max = torch.log(1 + (1 / (min_distances + model.module.epsilon)))
#                 prototype_activations_avg = torch.log(1 + (1 / (avg_distances + model.module.epsilon)))
#                 # print("prototype_activations_max", prototype_activations_max.shape)
#                 prototype_activations = prototype_activations_max - prototype_activations_avg
#                 logits = getattr(model.module, node.name + "_layer")(prototype_activations)
#                 setattr(node,"logits",logits)
#                 setattr(node,"min_distances",min_distances)

                pooled_node = pooled[node.name]#.squeeze(0)

                # get names specific to children
                children_idx = torch.tensor([name in node.descendents for name in batch_names])
                batch_names_coarsest = [node.closest_descendent_for(name).name for name in batch_names if name in node.descendents] # size of sum(children_idx)
                node_y = torch.tensor([node.children_to_labels[name] for name in batch_names_coarsest]).cuda() # size of sum(children_idx)
#                 node_logits = node.logits[children_idx]

                # if coarse_name (immediate child of node) itself has any children
                # list of [True, False] with the same size as batch_names_coarsest, 
                # True if the coarse_name has a child, Flase if not
                batch_names_coarsest_with_children_idx = [model.module.root.get_node(coarse_name).has_logits() for coarse_name in batch_names_coarsest] 
                batch_names_coarsest_with_children_idx = torch.tensor(batch_names_coarsest_with_children_idx)
                
                if batch_names_coarsest_with_children_idx.sum().item() == 0:
                    continue

                batch_names_coarsest_with_children = np.array(batch_names_coarsest)[batch_names_coarsest_with_children_idx.numpy()]
                
                batch_names_child_of_coarsest = [model.module.root.get_node(coarsest_name).closest_descendent_for(leaf_name).name \
                                                    for coarsest_name, leaf_name in zip(batch_names_coarsest_with_children, \
                                                                                        np.array(batch_names)[children_idx][batch_names_coarsest_with_children_idx.numpy()])]                
                
                pooled_node = pooled_node[children_idx][batch_names_coarsest_with_children_idx]
#                 prototype_activations_max = prototype_activations_max[children_idx][batch_names_coarsest_with_children_idx]

                num_protos_per_class = getattr(model.module.root, "num_prototypes_per_class")
                for i, (coarse_name, child_of_coarse_name) in enumerate(zip(batch_names_coarsest_with_children, batch_names_child_of_coarsest)):
                    for j in range(prototype_activations_max.shape[1]): # no.of prototypes
                        if node.children_to_labels[coarse_name] == int(j // num_protos_per_class):
                            max_activations["_".join([node.name, 'prototype_vectors', str(j)])][child_of_coarse_name].append(prototype_activations_max[i][j].item())
            except Exception as e:
                print(e)
                print(traceback.format_exc())
                pdb.set_trace()
# [model.module.root.get_node(coarsest_name).closest_descendent_for(leaf_name).name for coarsest_name, leaf_name in zip(batch_names_coarsest_with_children, np.array(batch_names)[children_idx][batch_names_coarsest_with_children_idx])]
# [coarsest_name for coarsest_name, leaf_name in zip(batch_names_coarsest_with_children, np.array(batch_names)[children_idx][batch_names_coarsest_with_children_idx])]

In [20]:
from util.data import ModifiedLabelLoader
from collections import defaultdict
import pdb

for node in root.nodes_with_children():
    if node.name == 'root':
        continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

    name2label = projectloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(projectloader, node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> (mean_activation, num_images)
    proto_mean_activations = defaultdict(lambda: defaultdict(lambda: [0, 0]))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        if coarse_label2name[ys.item()] not in non_leaf_children_names:
            continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            pfs, pooled, _ = net(xs, inference=True)
            pooled = pooled[node.name].squeeze(0) 
            pfs = pfs[node.name].squeeze(0)

            

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                if (len(relevant_proto_class_names) == 1) and (relevant_proto_class_names[0] not in non_leaf_children_names):
                    continue
                
                if (len(relevant_proto_class_names) == 1) and (coarse_label2name[ys.item()] in relevant_proto_class_names):
                    child_node = root.get_node(coarse_label2name[ys.item()])
                    grand_child = child_node.closest_descendent_for(label2name[orig_y.item()])
                    proto_mean_activations[p][grand_child.name][0] = ((proto_mean_activations[p][grand_child.name][0] * \
                                                                      proto_mean_activations[p][grand_child.name][1]) + pooled[p]) / (proto_mean_activations[p][grand_child.name][1] + 1)
                    proto_mean_activations[p][grand_child.name][1] += 1

                if (len(relevant_proto_class_names) > 1) and (coarse_label2name[ys.item()] in relevant_proto_class_names):
                    child_node = root.get_node(coarse_label2name[ys.item()])
                    if child_node.is_leaf():
                        proto_mean_activations[p][child_node.name][0] = ((proto_mean_activations[p][child_node.name][0] * \
                                                                            proto_mean_activations[p][child_node.name][1]) + pooled[p]) / (proto_mean_activations[p][child_node.name][1] + 1)
                        proto_mean_activations[p][child_node.name][1] += 1
                    else:
                        grand_child = child_node.closest_descendent_for(label2name[orig_y.item()])
                        proto_mean_activations[p][grand_child.name][0] = ((proto_mean_activations[p][grand_child.name][0] * \
                                                                            proto_mean_activations[p][grand_child.name][1]) + pooled[p]) / (proto_mean_activations[p][grand_child.name][1] + 1)
                        proto_mean_activations[p][grand_child.name][1] += 1
                

                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)

            # class_and_prototypes = defaultdict(list)
            # for p in range(pooled.shape[0]):
            #     class_and_prototypes[', '.join(list(proto_mean_activations[p].keys()))].append(p)
    
    print('Node', node.name)
    for child_classname in class_and_prototypes:
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            logstr = '\t'*2 + f'Proto:{p} '
            for grand_child_name in proto_mean_activations[p]:
                mean_activation = round(proto_mean_activations[p][grand_child_name][0].item(), 2)
                num_images = proto_mean_activations[p][grand_child_name][1]
                logstr += f'{grand_child_name}:{mean_activation}/{num_images} '
            print(logstr)


052+004
	 052+053
		Proto:0 053+051:0.42/90 cub_052_Pied_billed_Grebe:0.0/30 
		Proto:36 053+051:0.08/90 cub_052_Pied_billed_Grebe:0.29/30 
		Proto:5 053+051:0.17/90 cub_052_Pied_billed_Grebe:0.16/30 
		Proto:43 053+051:0.32/90 cub_052_Pied_billed_Grebe:0.37/30 
		Proto:13 053+051:0.38/90 cub_052_Pied_billed_Grebe:0.49/30 
		Proto:46 053+051:0.47/90 cub_052_Pied_billed_Grebe:0.01/30 
		Proto:16 053+051:0.46/90 cub_052_Pied_billed_Grebe:0.0/30 
		Proto:17 053+051:0.34/90 cub_052_Pied_billed_Grebe:0.03/30 
		Proto:23 053+051:0.03/90 cub_052_Pied_billed_Grebe:0.46/30 
		Proto:31 053+051:0.2/90 cub_052_Pied_billed_Grebe:0.06/30 
	 004+086
		Proto:2 086+045:0.18/300 004+032:0.12/120 
		Proto:3 086+045:0.19/300 004+032:0.16/120 
		Proto:4 086+045:0.32/300 004+032:0.18/120 
		Proto:7 086+045:0.19/300 004+032:0.35/120 
		Proto:8 086+045:0.12/300 004+032:0.24/120 
		Proto:10 086+045:0.0/300 004+032:0.51/120 
		Proto:11 086+045:0.27/300 004+032:0.06/120 
		Proto:12 086+045:0.03/300 004+032:0.35/

052+053
	 053+051
		Proto:2 051+050:0.49/60 cub_053_Western_Grebe:0.06/30 
		Proto:4 051+050:0.26/60 cub_053_Western_Grebe:0.39/30 
		Proto:5 051+050:0.11/60 cub_053_Western_Grebe:0.1/30 
		Proto:6 051+050:0.38/60 cub_053_Western_Grebe:0.72/30 
		Proto:7 051+050:0.17/60 cub_053_Western_Grebe:0.63/30 
		Proto:9 051+050:0.42/60 cub_053_Western_Grebe:0.83/30 
		Proto:10 051+050:0.14/60 cub_053_Western_Grebe:0.74/30 
		Proto:11 051+050:0.69/60 cub_053_Western_Grebe:0.67/30 
		Proto:15 051+050:0.4/60 cub_053_Western_Grebe:0.9/30 
		Proto:16 051+050:0.52/60 cub_053_Western_Grebe:0.43/30 
		Proto:17 051+050:0.74/60 cub_053_Western_Grebe:0.46/30 
		Proto:18 051+050:0.47/60 cub_053_Western_Grebe:0.24/30 
		Proto:20 051+050:0.18/60 cub_053_Western_Grebe:0.62/30 
		Proto:21 051+050:0.19/60 cub_053_Western_Grebe:0.26/30 
		Proto:22 051+050:0.35/60 cub_053_Western_Grebe:0.07/30 
		Proto:23 051+050:0.31/60 cub_053_Western_Grebe:0.01/30 
		Proto:24 051+050:0.41/60 cub_053_Western_Grebe:0.58/30 
		Pro

004+086
	 086+045
		Proto:1 045+100:0.18/270 cub_086_Pacific_Loon:0.48/30 
		Proto:4 045+100:0.4/270 cub_086_Pacific_Loon:0.33/30 
		Proto:5 045+100:0.13/270 cub_086_Pacific_Loon:0.0/30 
		Proto:10 045+100:0.46/270 cub_086_Pacific_Loon:0.53/30 
		Proto:11 045+100:0.31/270 cub_086_Pacific_Loon:0.01/30 
		Proto:13 045+100:0.31/270 cub_086_Pacific_Loon:0.0/30 
		Proto:14 045+100:0.16/270 cub_086_Pacific_Loon:0.27/30 
		Proto:17 045+100:0.45/270 cub_086_Pacific_Loon:0.08/30 
		Proto:19 045+100:0.24/270 cub_086_Pacific_Loon:0.0/30 
		Proto:21 045+100:0.12/270 cub_086_Pacific_Loon:0.25/30 
		Proto:28 045+100:0.23/270 cub_086_Pacific_Loon:0.35/30 
		Proto:29 045+100:0.15/270 cub_086_Pacific_Loon:0.28/30 
		Proto:32 045+100:0.32/270 cub_086_Pacific_Loon:0.6/30 
		Proto:38 045+100:0.21/270 cub_086_Pacific_Loon:0.01/30 
		Proto:39 045+100:0.29/270 cub_086_Pacific_Loon:0.03/30 
		Proto:40 045+100:0.16/270 cub_086_Pacific_Loon:0.2/30 
		Proto:41 045+100:0.15/270 cub_086_Pacific_Loon:0.28/30 
		Pro

053+051
	 051+050
		Proto:3 cub_050_Eared_Grebe:0.21/30 cub_051_Horned_Grebe:0.4/30 
		Proto:5 cub_050_Eared_Grebe:0.05/30 cub_051_Horned_Grebe:0.2/30 
		Proto:8 cub_050_Eared_Grebe:0.11/30 cub_051_Horned_Grebe:0.23/30 
		Proto:9 cub_050_Eared_Grebe:0.57/30 cub_051_Horned_Grebe:0.6/30 
		Proto:10 cub_050_Eared_Grebe:0.46/30 cub_051_Horned_Grebe:0.62/30 
		Proto:11 cub_050_Eared_Grebe:0.41/30 cub_051_Horned_Grebe:0.5/30 
		Proto:12 cub_050_Eared_Grebe:0.74/30 cub_051_Horned_Grebe:0.73/30 
		Proto:13 cub_050_Eared_Grebe:0.58/30 cub_051_Horned_Grebe:0.58/30 
		Proto:16 cub_050_Eared_Grebe:0.55/30 cub_051_Horned_Grebe:0.39/30 
		Proto:17 cub_050_Eared_Grebe:0.5/30 cub_051_Horned_Grebe:0.63/30 
		Proto:18 cub_050_Eared_Grebe:0.22/30 cub_051_Horned_Grebe:0.45/30 
		Proto:20 cub_050_Eared_Grebe:0.16/30 cub_051_Horned_Grebe:0.3/30 
		Proto:25 cub_050_Eared_Grebe:0.27/30 cub_051_Horned_Grebe:0.47/30 
		Proto:32 cub_050_Eared_Grebe:0.33/30 cub_051_Horned_Grebe:0.34/30 
		Proto:36 cub_050_Eared_G

004+032
	 032+031
		Proto:0 031+033:0.2/60 cub_032_Mangrove_Cuckoo:0.69/30 
		Proto:1 031+033:0.62/60 cub_032_Mangrove_Cuckoo:0.51/30 
		Proto:4 031+033:0.5/60 cub_032_Mangrove_Cuckoo:0.13/30 
		Proto:5 031+033:0.56/60 cub_032_Mangrove_Cuckoo:0.57/30 
		Proto:6 031+033:0.52/60 cub_032_Mangrove_Cuckoo:0.39/30 
		Proto:7 031+033:0.31/60 cub_032_Mangrove_Cuckoo:0.35/30 
		Proto:9 031+033:0.6/60 cub_032_Mangrove_Cuckoo:0.4/30 
		Proto:12 031+033:0.64/60 cub_032_Mangrove_Cuckoo:0.61/30 
		Proto:16 031+033:0.35/60 cub_032_Mangrove_Cuckoo:0.72/30 
		Proto:17 031+033:0.32/60 cub_032_Mangrove_Cuckoo:0.23/30 
		Proto:18 031+033:0.58/60 cub_032_Mangrove_Cuckoo:0.14/30 
		Proto:19 031+033:0.36/60 cub_032_Mangrove_Cuckoo:0.41/30 
		Proto:20 031+033:0.48/60 cub_032_Mangrove_Cuckoo:0.37/30 
		Proto:22 031+033:0.54/60 cub_032_Mangrove_Cuckoo:0.22/30 
		Proto:27 031+033:0.42/60 cub_032_Mangrove_Cuckoo:0.44/30 
		Proto:30 031+033:0.07/60 cub_032_Mangrove_Cuckoo:0.14/30 
		Proto:31 031+033:0.31/60 cub_03

...

045+100
	 100+023
		Proto:32 023+024:0.1/90 100+101:0.6/60 
		Proto:2 023+024:0.56/90 100+101:0.03/60 
		Proto:35 023+024:0.03/90 100+101:0.81/60 
		Proto:6 023+024:0.22/90 100+101:0.4/60 
		Proto:7 023+024:0.68/90 100+101:0.01/60 
		Proto:8 023+024:0.36/90 100+101:0.01/60 
		Proto:40 023+024:0.49/90 100+101:0.21/60 
		Proto:41 023+024:0.21/90 100+101:0.3/60 
		Proto:11 023+024:0.29/90 100+101:0.14/60 
		Proto:44 023+024:0.49/90 100+101:0.0/60 
		Proto:14 023+024:0.25/90 100+101:0.05/60 
		Proto:46 023+024:0.3/90 100+101:0.18/60 
		Proto:19 023+024:0.26/90 100+101:0.19/60 
		Proto:22 023+024:0.4/90 100+101:0.57/60 
		Proto:25 023+024:0.42/90 100+101:0.03/60 
		Proto:27 023+024:0.38/90 100+101:0.68/60 
	 045+003
		Proto:33 003+001:0.25/90 cub_045_Northern_Fulmar:0.46/30 
		Proto:34 003+001:0.13/90 cub_045_Northern_Fulmar:0.27/30 
		Proto:3 003+001:0.23/90 cub_045_Northern_Fulmar:0.47/30 
		Proto:36 003+001:0.5/90 cub_045_Northern_Fulmar:0.53/30 
		Proto:5 003+001:0.39/90 cub_045_Norther

045+003
	 003+001
		Proto:0 001+002:0.57/60 cub_003_Sooty_Albatross:0.4/30 
		Proto:1 001+002:0.7/60 cub_003_Sooty_Albatross:0.24/30 
		Proto:6 001+002:0.25/60 cub_003_Sooty_Albatross:0.73/30 
		Proto:8 001+002:0.31/60 cub_003_Sooty_Albatross:0.46/30 
		Proto:10 001+002:0.62/60 cub_003_Sooty_Albatross:0.38/30 
		Proto:13 001+002:0.37/60 cub_003_Sooty_Albatross:0.51/30 
		Proto:14 001+002:0.34/60 cub_003_Sooty_Albatross:0.49/30 
		Proto:17 001+002:0.58/60 cub_003_Sooty_Albatross:0.35/30 
		Proto:21 001+002:0.32/60 cub_003_Sooty_Albatross:0.67/30 
		Proto:24 001+002:0.31/60 cub_003_Sooty_Albatross:0.51/30 
		Proto:27 001+002:0.52/60 cub_003_Sooty_Albatross:0.34/30 
		Proto:28 001+002:0.65/60 cub_003_Sooty_Albatross:0.3/30 
		Proto:29 001+002:0.4/60 cub_003_Sooty_Albatross:0.17/30 
		Proto:30 001+002:0.71/60 cub_003_Sooty_Albatross:0.41/30 
		Proto:31 001+002:0.49/60 cub_003_Sooty_Albatross:0.28/30 
		Proto:37 001+002:0.43/60 cub_003_Sooty_Albatross:0.32/30 
		Proto:40 001+002:0.48/60 cub

100+023
	 023+024
		Proto:0 cub_023_Brandt_Cormorant:0.11/30 024+025:0.62/60 
		Proto:2 cub_023_Brandt_Cormorant:0.56/30 024+025:0.55/60 
		Proto:3 cub_023_Brandt_Cormorant:0.51/30 024+025:0.62/60 
		Proto:4 cub_023_Brandt_Cormorant:0.5/30 024+025:0.58/60 
		Proto:35 cub_023_Brandt_Cormorant:0.27/30 024+025:0.66/60 
		Proto:7 cub_023_Brandt_Cormorant:0.56/30 024+025:0.71/60 
		Proto:41 cub_023_Brandt_Cormorant:0.47/30 024+025:0.57/60 
		Proto:42 cub_023_Brandt_Cormorant:0.13/30 024+025:0.54/60 
		Proto:43 cub_023_Brandt_Cormorant:0.47/30 024+025:0.52/60 
		Proto:12 cub_023_Brandt_Cormorant:0.32/30 024+025:0.32/60 
		Proto:46 cub_023_Brandt_Cormorant:0.39/30 024+025:0.54/60 
		Proto:16 cub_023_Brandt_Cormorant:0.51/30 024+025:0.53/60 
		Proto:48 cub_023_Brandt_Cormorant:0.4/30 024+025:0.46/60 
		Proto:18 cub_023_Brandt_Cormorant:0.27/30 024+025:0.31/60 
		Proto:20 cub_023_Brandt_Cormorant:0.56/30 024+025:0.49/60 
		Proto:24 cub_023_Brandt_Cormorant:0.67/30 024+025:0.4/60 
		Proto:28 cub

003+001
	 001+002
		Proto:0 cub_001_Black_footed_Albatross:0.71/30 cub_002_Laysan_Albatross:0.68/30 
		Proto:33 cub_001_Black_footed_Albatross:0.55/30 cub_002_Laysan_Albatross:0.42/30 
		Proto:3 cub_001_Black_footed_Albatross:0.46/30 cub_002_Laysan_Albatross:0.54/30 
		Proto:36 cub_001_Black_footed_Albatross:0.66/30 cub_002_Laysan_Albatross:0.43/30 
		Proto:6 cub_001_Black_footed_Albatross:0.47/30 cub_002_Laysan_Albatross:0.52/30 
		Proto:7 cub_001_Black_footed_Albatross:0.46/30 cub_002_Laysan_Albatross:0.63/30 
		Proto:40 cub_001_Black_footed_Albatross:0.37/30 cub_002_Laysan_Albatross:0.63/30 
		Proto:9 cub_001_Black_footed_Albatross:0.57/30 cub_002_Laysan_Albatross:0.65/30 
		Proto:42 cub_001_Black_footed_Albatross:0.68/30 cub_002_Laysan_Albatross:0.46/30 
		Proto:12 cub_001_Black_footed_Albatross:0.76/30 cub_002_Laysan_Albatross:0.78/30 
		Proto:17 cub_001_Black_footed_Albatross:0.8/30 cub_002_Laysan_Albatross:0.57/30 
		Proto:24 cub_001_Black_footed_Albatross:0.64/30 cub_002_Laysan

023+024
	 024+025
		Proto:33 cub_024_Red_faced_Cormorant:0.64/30 cub_025_Pelagic_Cormorant:0.31/30 
		Proto:35 cub_024_Red_faced_Cormorant:0.32/30 cub_025_Pelagic_Cormorant:0.48/30 
		Proto:37 cub_024_Red_faced_Cormorant:0.67/30 cub_025_Pelagic_Cormorant:0.52/30 
		Proto:38 cub_024_Red_faced_Cormorant:0.69/30 cub_025_Pelagic_Cormorant:0.41/30 
		Proto:7 cub_024_Red_faced_Cormorant:0.84/30 cub_025_Pelagic_Cormorant:0.32/30 
		Proto:9 cub_024_Red_faced_Cormorant:0.85/30 cub_025_Pelagic_Cormorant:0.15/30 
		Proto:10 cub_024_Red_faced_Cormorant:0.48/30 cub_025_Pelagic_Cormorant:0.44/30 
		Proto:12 cub_024_Red_faced_Cormorant:0.84/30 cub_025_Pelagic_Cormorant:0.5/30 
		Proto:13 cub_024_Red_faced_Cormorant:0.48/30 cub_025_Pelagic_Cormorant:0.26/30 
		Proto:14 cub_024_Red_faced_Cormorant:0.45/30 cub_025_Pelagic_Cormorant:0.55/30 
		Proto:47 cub_024_Red_faced_Cormorant:0.5/30 cub_025_Pelagic_Cormorant:0.32/30 
		Proto:18 cub_024_Red_faced_Cormorant:0.76/30 cub_025_Pelagic_Cormorant:0.26/30 
		

In [24]:
from util.data import ModifiedLabelLoader
from collections import defaultdict
import pdb

for node in root.nodes_with_children():
    if node.name == 'root':
        continue
    non_leaf_children_names = [child.name for child in node.children if not child.is_leaf()]
    if len(non_leaf_children_names) == 0: # if all the children are leaf nodes then skip this node
        continue

    name2label = projectloader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    modifiedLabelLoader = ModifiedLabelLoader(projectloader, node)
    coarse_label2name = modifiedLabelLoader.modifiedlabel2name
    node_label_to_children = {label: name for name, label in node.children_to_labels.items()}

    img_iter = tqdm(enumerate(modifiedLabelLoader),
                    total=len(modifiedLabelLoader),
                    mininterval=50.,
                    desc='Collecting topk',
                    ncols=0)

    

    classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
    
    # maps proto_number -> grand_child_name (or descendant leaf name) -> (mean_activation, num_images)
    proto_mean_activations = defaultdict(lambda: defaultdict(lambda: [0, 0]))

    # maps class names to the prototypes that belong to that
    class_and_prototypes = defaultdict(set)

    for i, (xs, orig_y, ys) in img_iter:
        if coarse_label2name[ys.item()] not in non_leaf_children_names:
            continue

        xs, ys = xs.to(device), ys.to(device)

        with torch.no_grad():
            pfs, pooled, _ = net(xs, inference=True)
            pooled = pooled[node.name].squeeze(0) 
            pfs = pfs[node.name].squeeze(0)

            for p in range(pooled.shape[0]): # pooled.shape -> [768] (== num of prototypes)
                c_weight = torch.max(classification_weights[:,p]) # classification_weights[:,p].shape -> [200] (== num of classes)
                relevant_proto_classes = torch.nonzero(classification_weights[:, p] > 1e-3)
                relevant_proto_class_names = [node_label_to_children[class_idx.item()] for class_idx in relevant_proto_classes]

                if len(relevant_proto_class_names) == 0:
                    continue
                
                if (len(relevant_proto_class_names) == 1) and (relevant_proto_class_names[0] not in non_leaf_children_names):
                    continue
                
                if (coarse_label2name[ys.item()] in relevant_proto_class_names):
                    child_node = root.get_node(coarse_label2name[ys.item()])
                    leaf_descendent = label2name[orig_y.item()][4:7]
                    proto_mean_activations[p][leaf_descendent][0] = ((proto_mean_activations[p][leaf_descendent][0] * \
                                                                      proto_mean_activations[p][leaf_descendent][1]) + pooled[p]) / (proto_mean_activations[p][leaf_descendent][1] + 1)
                    proto_mean_activations[p][leaf_descendent][1] += 1                

                class_and_prototypes[', '.join(relevant_proto_class_names)].add(p)

    
    print('Node', node.name)
    for child_classname in class_and_prototypes:
        print('\t'*1, 'Child:', child_classname)
        for p in class_and_prototypes[child_classname]:
            logstr = '\t'*2 + f'Proto:{p} '
            for leaf_descendent in proto_mean_activations[p]:
                mean_activation = round(proto_mean_activations[p][leaf_descendent][0].item(), 2)
#                 num_images = proto_mean_activations[p][leaf_descendent][1]
#                 logstr += f'{leaf_descendent}:{mean_activation}/{num_images} '
                num_images = proto_mean_activations[p][leaf_descendent][1]
                logstr += f'{leaf_descendent}:({mean_activation}) '
            print(logstr)


Node 052+004
	 Child: 052+053
		Proto:0 050:(0.23) 051:(0.57) 052:(0.0) 053:(0.45) 
		Proto:36 050:(0.16) 051:(0.15) 052:(0.31) 053:(0.0) 
		Proto:5 050:(0.12) 051:(0.4) 052:(0.14) 053:(0.0) 
		Proto:43 050:(0.2) 051:(0.52) 052:(0.35) 053:(0.18) 
		Proto:13 050:(0.36) 051:(0.46) 052:(0.54) 053:(0.3) 
		Proto:46 050:(0.43) 051:(0.46) 052:(0.02) 053:(0.49) 
		Proto:16 050:(0.17) 051:(0.52) 052:(0.0) 053:(0.64) 
		Proto:17 050:(0.12) 051:(0.44) 052:(0.03) 053:(0.48) 
		Proto:23 050:(0.01) 051:(0.04) 052:(0.42) 053:(0.06) 
		Proto:31 050:(0.16) 051:(0.38) 052:(0.06) 053:(0.0) 
	 Child: 004+086
		Proto:2 001:(0.15) 002:(0.27) 003:(0.23) 004:(0.23) 023:(0.16) 024:(0.18) 025:(0.34) 031:(0.13) 032:(0.07) 033:(0.06) 045:(0.11) 086:(0.12) 100:(0.11) 101:(0.11) 
		Proto:3 001:(0.2) 002:(0.47) 003:(0.64) 004:(0.03) 023:(0.0) 024:(0.01) 025:(0.03) 031:(0.34) 032:(0.07) 033:(0.28) 045:(0.43) 086:(0.01) 100:(0.03) 101:(0.09) 
		Proto:4 001:(0.66) 002:(0.03) 003:(0.31) 004:(0.03) 023:(0.56) 024:(0.13)

Node 052+053
	 Child: 053+051
		Proto:2 050:(0.5) 051:(0.54) 053:(0.04) 
		Proto:4 050:(0.17) 051:(0.33) 053:(0.36) 
		Proto:5 050:(0.08) 051:(0.12) 053:(0.1) 
		Proto:6 050:(0.35) 051:(0.34) 053:(0.7) 
		Proto:7 050:(0.01) 051:(0.31) 053:(0.66) 
		Proto:9 050:(0.38) 051:(0.44) 053:(0.81) 
		Proto:10 050:(0.12) 051:(0.21) 053:(0.68) 
		Proto:11 050:(0.68) 051:(0.71) 053:(0.62) 
		Proto:15 050:(0.49) 051:(0.3) 053:(0.87) 
		Proto:16 050:(0.52) 051:(0.54) 053:(0.36) 
		Proto:17 050:(0.72) 051:(0.8) 053:(0.38) 
		Proto:18 050:(0.39) 051:(0.47) 053:(0.22) 
		Proto:20 050:(0.01) 051:(0.35) 053:(0.57) 
		Proto:21 050:(0.23) 051:(0.16) 053:(0.22) 
		Proto:22 050:(0.36) 051:(0.4) 053:(0.08) 
		Proto:23 050:(0.18) 051:(0.41) 053:(0.01) 
		Proto:24 050:(0.4) 051:(0.44) 053:(0.55) 
		Proto:26 050:(0.31) 051:(0.39) 053:(0.45) 
		Proto:27 050:(0.35) 051:(0.46) 053:(0.5) 
		Proto:34 050:(0.43) 051:(0.47) 053:(0.18) 
		Proto:36 050:(0.5) 051:(0.44) 053:(0.77) 
		Proto:38 050:(0.01) 051:(0.24) 053:(0.

Node 004+086
	 Child: 086+045
		Proto:1 001:(0.0) 002:(0.0) 003:(0.05) 023:(0.59) 024:(0.12) 025:(0.47) 045:(0.0) 086:(0.51) 100:(0.26) 101:(0.02) 
		Proto:4 001:(0.49) 002:(0.65) 003:(0.23) 023:(0.15) 024:(0.74) 025:(0.23) 045:(0.04) 086:(0.31) 100:(0.49) 101:(0.71) 
		Proto:5 001:(0.04) 002:(0.0) 003:(0.14) 023:(0.11) 024:(0.59) 025:(0.18) 045:(0.0) 086:(0.0) 100:(0.05) 101:(0.0) 
		Proto:10 001:(0.49) 002:(0.68) 003:(0.25) 023:(0.39) 024:(0.56) 025:(0.32) 045:(0.01) 086:(0.41) 100:(0.64) 101:(0.6) 
		Proto:11 001:(0.3) 002:(0.66) 003:(0.3) 023:(0.08) 024:(0.12) 025:(0.15) 045:(0.36) 086:(0.01) 100:(0.07) 101:(0.59) 
		Proto:13 001:(0.06) 002:(0.55) 003:(0.06) 023:(0.02) 024:(0.38) 025:(0.0) 045:(0.28) 086:(0.0) 100:(0.66) 101:(0.75) 
		Proto:14 001:(0.24) 002:(0.13) 003:(0.33) 023:(0.3) 024:(0.15) 025:(0.19) 045:(0.03) 086:(0.32) 100:(0.0) 101:(0.0) 
		Proto:17 001:(0.17) 002:(0.69) 003:(0.11) 023:(0.17) 024:(0.84) 025:(0.21) 045:(0.48) 086:(0.08) 100:(0.81) 101:(0.66) 
		Proto:19 0

Node 053+051
	 Child: 051+050
		Proto:3 050:(0.19) 051:(0.38) 
		Proto:5 050:(0.06) 051:(0.22) 
		Proto:8 050:(0.09) 051:(0.25) 
		Proto:9 050:(0.55) 051:(0.62) 
		Proto:10 050:(0.46) 051:(0.59) 
		Proto:11 050:(0.38) 051:(0.49) 
		Proto:12 050:(0.73) 051:(0.73) 
		Proto:13 050:(0.59) 051:(0.58) 
		Proto:16 050:(0.56) 051:(0.35) 
		Proto:17 050:(0.46) 051:(0.61) 
		Proto:18 050:(0.28) 051:(0.45) 
		Proto:20 050:(0.17) 051:(0.28) 
		Proto:25 050:(0.26) 051:(0.46) 
		Proto:32 050:(0.32) 051:(0.32) 
		Proto:36 050:(0.39) 051:(0.48) 
		Proto:38 050:(0.37) 051:(0.47) 
		Proto:42 050:(0.39) 051:(0.35) 
		Proto:43 050:(0.36) 051:(0.41) 
		Proto:45 050:(0.2) 051:(0.45) 
		Proto:47 050:(0.51) 051:(0.62) 
		Proto:48 050:(0.29) 051:(0.34) 
		Proto:49 050:(0.57) 051:(0.64) 


Node 004+032
	 Child: 032+031
		Proto:0 031:(0.28) 032:(0.65) 033:(0.22) 
		Proto:1 031:(0.77) 032:(0.49) 033:(0.52) 
		Proto:4 031:(0.48) 032:(0.13) 033:(0.54) 
		Proto:5 031:(0.36) 032:(0.49) 033:(0.75) 
		Proto:6 031:(0.52) 032:(0.38) 033:(0.5) 
		Proto:7 031:(0.0) 032:(0.36) 033:(0.67) 
		Proto:9 031:(0.61) 032:(0.4) 033:(0.62) 
		Proto:12 031:(0.34) 032:(0.59) 033:(0.82) 
		Proto:16 031:(0.21) 032:(0.72) 033:(0.54) 
		Proto:17 031:(0.38) 032:(0.23) 033:(0.24) 
		Proto:18 031:(0.69) 032:(0.16) 033:(0.52) 
		Proto:19 031:(0.43) 032:(0.4) 033:(0.3) 
		Proto:20 031:(0.44) 032:(0.36) 033:(0.46) 
		Proto:22 031:(0.65) 032:(0.23) 033:(0.46) 
		Proto:27 031:(0.39) 032:(0.41) 033:(0.4) 
		Proto:30 031:(0.13) 032:(0.12) 033:(0.09) 
		Proto:31 031:(0.2) 032:(0.34) 033:(0.4) 
		Proto:32 031:(0.73) 032:(0.39) 033:(0.63) 
		Proto:36 031:(0.46) 032:(0.08) 033:(0.35) 
		Proto:37 031:(0.12) 032:(0.27) 033:(0.63) 
		Proto:38 031:(0.42) 032:(0.51) 033:(0.46) 
		Proto:40 031:(0.88) 032:(0.43) 033:(0.

...

Node 045+100
	 Child: 100+023
		Proto:32 023:(0.01) 024:(0.16) 025:(0.08) 100:(0.63) 101:(0.6) 
		Proto:2 023:(0.4) 024:(0.83) 025:(0.43) 100:(0.02) 101:(0.04) 
		Proto:35 023:(0.03) 024:(0.0) 025:(0.05) 100:(0.85) 101:(0.81) 
		Proto:6 023:(0.0) 024:(0.69) 025:(0.02) 100:(0.44) 101:(0.4) 
		Proto:7 023:(0.67) 024:(0.74) 025:(0.64) 100:(0.01) 101:(0.02) 
		Proto:8 023:(0.31) 024:(0.41) 025:(0.35) 100:(0.06) 101:(0.0) 
		Proto:40 023:(0.24) 024:(0.77) 025:(0.5) 100:(0.2) 101:(0.17) 
		Proto:41 023:(0.0) 024:(0.57) 025:(0.08) 100:(0.09) 101:(0.5) 
		Proto:11 023:(0.04) 024:(0.81) 025:(0.0) 100:(0.08) 101:(0.17) 
		Proto:44 023:(0.61) 024:(0.63) 025:(0.2) 100:(0.02) 101:(0.0) 
		Proto:14 023:(0.45) 024:(0.18) 025:(0.11) 100:(0.12) 101:(0.0) 
		Proto:46 023:(0.45) 024:(0.04) 025:(0.46) 100:(0.4) 101:(0.03) 
		Proto:19 023:(0.01) 024:(0.73) 025:(0.04) 100:(0.07) 101:(0.31) 
		Proto:22 023:(0.47) 024:(0.24) 025:(0.52) 100:(0.42) 101:(0.72) 
		Proto:25 023:(0.18) 024:(0.38) 025:(0.7) 100:(0.0

Node 045+003
	 Child: 003+001
		Proto:0 001:(0.64) 002:(0.51) 003:(0.45) 
		Proto:1 001:(0.73) 002:(0.6) 003:(0.29) 
		Proto:6 001:(0.18) 002:(0.23) 003:(0.81) 
		Proto:8 001:(0.39) 002:(0.29) 003:(0.44) 
		Proto:10 001:(0.53) 002:(0.6) 003:(0.34) 
		Proto:13 001:(0.35) 002:(0.32) 003:(0.55) 
		Proto:14 001:(0.45) 002:(0.18) 003:(0.44) 
		Proto:17 001:(0.62) 002:(0.55) 003:(0.33) 
		Proto:21 001:(0.12) 002:(0.59) 003:(0.67) 
		Proto:24 001:(0.25) 002:(0.34) 003:(0.46) 
		Proto:27 001:(0.57) 002:(0.49) 003:(0.33) 
		Proto:28 001:(0.46) 002:(0.75) 003:(0.34) 
		Proto:29 001:(0.36) 002:(0.36) 003:(0.2) 
		Proto:30 001:(0.77) 002:(0.62) 003:(0.34) 
		Proto:31 001:(0.42) 002:(0.52) 003:(0.27) 
		Proto:37 001:(0.38) 002:(0.45) 003:(0.34) 
		Proto:40 001:(0.45) 002:(0.55) 003:(0.33) 
		Proto:41 001:(0.3) 002:(0.83) 003:(0.22) 
		Proto:46 001:(0.43) 002:(0.54) 003:(0.13) 
		Proto:47 001:(0.71) 002:(0.16) 003:(0.13) 


Node 100+023
	 Child: 023+024
		Proto:0 023:(0.09) 024:(0.8) 025:(0.35) 
		Proto:2 023:(0.54) 024:(0.54) 025:(0.56) 
		Proto:3 023:(0.49) 024:(0.67) 025:(0.6) 
		Proto:4 023:(0.47) 024:(0.52) 025:(0.6) 
		Proto:35 023:(0.29) 024:(0.8) 025:(0.49) 
		Proto:7 023:(0.58) 024:(0.9) 025:(0.56) 
		Proto:41 023:(0.44) 024:(0.62) 025:(0.49) 
		Proto:42 023:(0.13) 024:(0.51) 025:(0.55) 
		Proto:43 023:(0.46) 024:(0.47) 025:(0.53) 
		Proto:12 023:(0.32) 024:(0.58) 025:(0.07) 
		Proto:46 023:(0.37) 024:(0.6) 025:(0.5) 
		Proto:16 023:(0.48) 024:(0.83) 025:(0.27) 
		Proto:48 023:(0.43) 024:(0.36) 025:(0.58) 
		Proto:18 023:(0.24) 024:(0.43) 025:(0.21) 
		Proto:20 023:(0.56) 024:(0.37) 025:(0.61) 
		Proto:24 023:(0.64) 024:(0.37) 025:(0.47) 
		Proto:28 023:(0.3) 024:(0.65) 025:(0.32) 
		Proto:31 023:(0.67) 024:(0.73) 025:(0.3) 
	 Child: 100+101
		Proto:1 100:(0.57) 101:(0.49) 
		Proto:33 100:(0.47) 101:(0.51) 
		Proto:36 100:(0.78) 101:(0.6) 
		Proto:6 100:(0.56) 101:(0.54) 
		Proto:8 100:(0.81) 101

Node 003+001
	 Child: 001+002
		Proto:0 001:(0.71) 002:(0.65) 
		Proto:33 001:(0.56) 002:(0.48) 
		Proto:3 001:(0.46) 002:(0.59) 
		Proto:36 001:(0.66) 002:(0.42) 
		Proto:6 001:(0.5) 002:(0.5) 
		Proto:7 001:(0.43) 002:(0.66) 
		Proto:40 001:(0.35) 002:(0.66) 
		Proto:9 001:(0.54) 002:(0.65) 
		Proto:42 001:(0.6) 002:(0.45) 
		Proto:12 001:(0.77) 002:(0.81) 
		Proto:17 001:(0.78) 002:(0.58) 
		Proto:24 001:(0.6) 002:(0.61) 
		Proto:25 001:(0.47) 002:(0.35) 
		Proto:30 001:(0.55) 002:(0.56) 
		Proto:31 001:(0.67) 002:(0.65) 


Node 023+024
	 Child: 024+025
		Proto:33 024:(0.61) 025:(0.29) 
		Proto:35 024:(0.28) 025:(0.47) 
		Proto:37 024:(0.68) 025:(0.47) 
		Proto:38 024:(0.63) 025:(0.44) 
		Proto:7 024:(0.78) 025:(0.28) 
		Proto:9 024:(0.81) 025:(0.17) 
		Proto:10 024:(0.51) 025:(0.41) 
		Proto:12 024:(0.83) 025:(0.49) 
		Proto:13 024:(0.45) 025:(0.3) 
		Proto:14 024:(0.46) 025:(0.57) 
		Proto:47 024:(0.48) 025:(0.32) 
		Proto:18 024:(0.77) 025:(0.25) 
		Proto:26 024:(0.44) 025:(0.52) 
		Proto:27 024:(0.35) 025:(0.27) 
		Proto:31 024:(0.51) 025:(0.19) 
